In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)

print(sys.path)

%load_ext autoreload

%autoreload 2

from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels, nn_libs 

['', '/home/kai/anaconda3/envs/tf_gpu/lib/python35.zip', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/plat-linux', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/lib-dynload', '/home/kai/.local/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages', '/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/IPython/extensions', '/home/kai/.ipython', '/home/kai/data/shiyi/AlphaBoosting/automl']


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek']#, 'DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Distance']

In [3]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list):
    features_to_gen = []
    for function in function_list:
        for i in range(2, len(categorical_features)+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

#features_to_gen = get_features_to_gen([fe.count, fe.unique_count, fe.cumulative_count,
#                                      fe.reverse_cumulative_count, fe.variance, fe.count_std_over_mean])
features_to_gen = get_features_to_gen([fe.count])

features_to_gen

[{'feature_cols': ['Month', 'DayofMonth'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}},
 {'feature_cols': ['Month', 'DayofMonth', 'DayOfWeek'],
  'function': <function automl_libs.feature_engineering.count(df, cols, dummy_col, generated_feature_name, params=None)>,
  'params': {}}]

In [4]:
len(features_to_gen)

4

In [5]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/' 

In [6]:
from automl_app import logger_config

logger_config.config(project_path+'log_project1.log')

In [62]:
def gs_params_gen(model='lgb'):
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    if model == 'lgb':
        params = {    
            'objective': 'binary',
            'boosting': 'gbdt',
            'num_boost_round': 50, # ignored in params. extract it and put it in input arguments in train or cv explicitly
                                   # seems to work fine as the upper limit when combined with early_stopping_round
            'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
            'num_leaves': np.random.choice([15,31]),#,61,127]),
            'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
            'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
            'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
            'feature_fraction': np.random.randint(9,11)/10,
            'feature_fraction_seed': seed,
            'early_stopping_round':3,
            'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
            'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
            'bagging_seed': seed,
            'scale_pos_weight': 1,
            'metric' : 'auc' 
        }
    elif model == 'nn':
        params = {
            'nn_seed': int(time.time() * 1000000) % 45234634,
            'ep_for_lr': 1,
            'lr_init': 0.01,
            'lr_fin': 0.005, # if == lr_init, then no lr decay
            'batch_size': np.random.choice([32, 64, 128, 256, 512, 1024, 2048]),
            "pred_batch_size": 100000,
            'max_ep': 1,
            'patience': 5,
            'cat_emb_outdim': 50, # could be a constant or a dict (col name:embed out dim). e.g.:
                                  # embed_outdim = [3, 3, 8, 8, 3]
                                  # embed_outdim_dict = dict(zip(X_train.columns.values, embed_outdim))
            'dense_units': 50,
            'num_dense_n_layers': 1,
            'drop_rate': 0.2,
            'combined_dense_n_layers': 1
        }
    return params, seed

In [63]:
gs_params_gen('lgb')

({'bagging_fraction': 0.6,
  'bagging_freq': 1,
  'bagging_seed': 43981441,
  'boosting': 'gbdt',
  'early_stopping_round': 3,
  'feature_fraction': 0.9,
  'feature_fraction_seed': 43981441,
  'learning_rate': 0.03,
  'max_depth': -1,
  'metric': 'auc',
  'min_data_in_leaf': 21,
  'num_boost_round': 50,
  'num_leaves': 31,
  'num_threads': 8,
  'objective': 'binary',
  'scale_pos_weight': 1},
 43981441)

In [64]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'last_run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, gs_params_gen)

2018-06-28 18:54:25 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #66  | Run record file [./output/last_run_record.json] found. Continue from the previous run...
2018-06-28 18:54:25 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #95  | generate todo list
2018-06-28 18:54:25 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #111 | STAGE: FEATURE_ENGINEERING
2018-06-28 18:54:25 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #115 | STAGE: VALIDATION_DOWNSAMPLING_GEN
2018-06-28 18:54:25 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #119 | STAGE: CONCAT_DATA
2018-06-28 18:54:25 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #123 | STAGE: GRID_SEARCH


Training until validation scores don't improve for 3 rounds.
[20]	training's auc: 0.725345	valid_1's auc: 0.721444
[40]	training's auc: 0.731653	valid_1's auc: 0.727403


2018-06-28 18:54:33 | DEBUG | automl_libs.grid_search       | _lgb_gs    | #101 | [do_preds] is True, generating predictions ...
2018-06-28 18:54:33 | DEBUG | automl_libs.grid_search       | _lgb_gs    | #102 | Retrain model using best_round and all data...


Did not meet early stopping. Best iteration is:
[50]	training's auc: 0.734732	valid_1's auc: 0.730163
[10]	training's auc: 0.719989
[20]	training's auc: 0.725223
[30]	training's auc: 0.728637
[40]	training's auc: 0.731797
[50]	training's auc: 0.734655


2018-06-28 18:54:40 | INFO  | automl_libs.grid_search       | _lgb_gs    | #113 | LGB predictions(1530226467) saved in ./output/gs_saved_preds/.
2018-06-28 18:54:40 | DEBUG | automl_libs.grid_search       | gs         | #52  | ./output/lgb_grid_search.csv updated


...

Did not meet early stopping. Best iteration is:
[50]	training's auc: 0.746279	valid_1's auc: 0.743096
[10]	training's auc: 0.721854
[20]	training's auc: 0.73033
[30]	training's auc: 0.73632
[40]	training's auc: 0.742531
[50]	training's auc: 0.746537


2018-06-28 18:55:09 | INFO  | automl_libs.grid_search       | _lgb_gs    | #113 | LGB predictions(1530226495) saved in ./output/gs_saved_preds/.
2018-06-28 18:55:09 | DEBUG | automl_libs.grid_search       | gs         | #52  | ./output/lgb_grid_search.csv updated
2018-06-28 18:55:09 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #127 | STAGE: STACKNET
2018-06-28 18:55:09 | DEBUG | automl_libs.stacknet          | layer1     | #36  | data_id: flight_data_ordinal  
	x_train: (500000, 12)	x_test: (100000, 12)
	y_train type: <class 'dict'>
	compatible_models: {'LGB'}
 
2018-06-28 18:55:09 | DEBUG | automl_libs.stacknet          | layer1     | #56  | 1530224002 already processed in StackNet, so poped it from chosen_gs_results
2018-06-28 18:55:09 | DEBUG | automl_libs.stacknet          | layer1     | #56  | 1530224256 already processed in StackNet, so poped it from chosen_gs_results
2018-06-28 18:55:09 | DEBUG | automl_libs.stacknet          | layer1     | #62  | {'num_leaves': 31, 'm

load from file
Computing... label: label        model_data_id: 1530226495__LGB_flight_data_ordinal
processing fold 0...
No evaluation set, thus not possible to use early stopping. Please train with your best params.
[10]	training's auc: 0.702
[20]	training's auc: 0.706908
[30]	training's auc: 0.711612
[40]	training's auc: 0.718401
[50]	training's auc: 0.723574
processing fold 1...
No evaluation set, thus not possible to use early stopping. Please train with your best params.
[10]	training's auc: 0.701282
[20]	training's auc: 0.707007
[30]	training's auc: 0.712307
[40]	training's auc: 0.718261
[50]	training's auc: 0.723435
processing fold 2...
No evaluation set, thus not possible to use early stopping. Please train with your best params.
[10]	training's auc: 0.701466
[20]	training's auc: 0.707491
[30]	training's auc: 0.711287
[40]	training's auc: 0.717718
[50]	training's auc: 0.724038
processing fold 3...
No evaluation set, thus not possible to use early stopping. Please train with your

2018-06-28 18:55:48 | INFO  | automl_app.app.AlphaBoosting  | __init__   | #131 | save run record
2018-06-28 18:55:48 | INFO  | automl_app.app.AlphaBoosting  | _save_run_record | #163 | val index is saved at ./temp_data/val_index.pkl
2018-06-28 18:55:48 | INFO  | automl_app.app.AlphaBoosting  | _save_run_record | #165 | run record is saved at ./output/last_run_record.json


1530226536__LOGREG_layer2 0.7189749294878125
